In [70]:
import torch
import torch.nn as nn
import sys
import os

import numpy as np
import pandas as pd

src_dir = os.path.join(os.getcwd(), "src")
sys.path.append(os.path.abspath(src_dir))

In [71]:
from importlib import reload

import utils.data_loader
import models.resnet14
import utils.metrics
import utils.visualizer
import helpers.model_helper
import services.azure_bucket

reload(utils.data_loader)
reload(models.resnet14)
reload(utils.visualizer)
reload(utils.metrics)
reload(helpers.model_helper)
reload(services.azure_bucket)

from utils.data_loader import DeviceDataLoader
from utils.data_loader import DataPreparer
from models.resnet14 import ResNet14
from utils.visualizer import Visualizer
from utils.metrics import Metrics
from local_types.enums import EDataset

In [66]:
data_preparer = DataPreparer(image_size=75)
data_preparer.load_datasets(locally=True)

/home/dany_leguy/T-DEV-810-PAR_1/dataset/


Loading TEST: 100%|██████████████████████████████████| 585/585 [00:03<00:00, 151.99file/s]


In [67]:
device = DeviceDataLoader.get_default_device()
model = DeviceDataLoader.to_device(ResNet14(3,3), device)
model.load(
    file_name="resnet14_general_best_1.pth", 
    # path=os.path.join(os.getcwd(), "cloud/deployment/"), 
)

In [13]:
model.generate_tarball(
    initial_folder_path=os.path.join(os.getcwd(), "./cloud/deployment"),
    output_path=os.path.join(os.getcwd(), "src/processed"), 
)

In [72]:
visualizer = Visualizer()
visualizer.visualize_history_from_csv(
    file_name="resnet14_general_best_cross_val_2.csv", 
    have_loss=True, 
    have_score=True, 
    path=os.path.join(os.getcwd(), "src/processed/")
)

In [69]:
model.evaluate(model, data_preparer.get_dataloaders(
    dataset_type=EDataset.TEST,
    batch_size=10
))

{'val_loss': 0.48479732871055603,
 'val_acc': 0.820339024066925,
 'f1': 0.8063867567342723,
 'roc_auc': 0.9339222430685844}

In [56]:
Visualizer().visualize_several_csv(
    [
        "resnet14_general_best_1_SGD.csv", 
        "resnet14_general_best_1_Adam.csv", 
    ], 
    metric="val_loss", 
    column_names=[
        "SGD", 
        "Adam", 
    ],
    title="Image Size Comparison",
    x_name="Epoch",
    y_name="Validation Loss",
)